# Evolución del FIUBA map

En este caso, vamos a analizar cómo evoluciona la red del FIUBA map respecto a alumnos que alguna vez cursaron juntos.
El grafo que armaremos será de alumnos (nodos) que se conectan si alguna vez cursaron juntos una materia (aristas).

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import utils

In [ ]:
df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_rel = df[['Padron', 'materia_id', 'materia_cuatrimestre']].copy()

# Sacamos a quienes no ordenan su carrera por cuatrimestres
df_rel.dropna(subset=['materia_cuatrimestre'], inplace=True)

df_simil = pd.merge(df_rel, df_rel, on=['materia_id', 'materia_cuatrimestre'])
df_simil = df_simil[df_simil['Padron_x'] != df_simil['Padron_y']]
df_simil = df_simil[['Padron_x', 'Padron_y', 'materia_cuatrimestre']]
df_simil = df_simil.sort_values(by=['materia_cuatrimestre'])
df_simil.head(3)

In [ ]:
# tomamos los cuatrimestres hasta la actualidad, sin tener en cuenta cuatrimestres que no pasaron porque la cantidad de nodos se mantiene constante
cuatris = df_simil[df_simil['materia_cuatrimestre'] < 2023]['materia_cuatrimestre'].unique()
cuatris

In [ ]:
graphs = []
for cuatri in cuatris:
    df_simil_cuatri = df_simil[df_simil['materia_cuatrimestre'] <= cuatri]
    G = nx.from_pandas_edgelist(df_simil_cuatri, 
                                source='Padron_x', 
                                target='Padron_y',
                                create_using=nx.Graph())
    graphs.append((cuatri, G))
    print(f"""cuatri: {cuatri} - nodos: {len(G)} - aristas: {len(G.edges)} - diametro: {nx.diameter(G)} - grado promedio: {sum([n[1] for n in G.degree()]) / len(G):.2f}""")

# fig, axs = plt.subplots(ncols=2, nrows=len(graphs)//2, figsize=(20,50))
fig, axs = plt.subplots(ncols=7, nrows=len(graphs)//7, figsize=(20,5))
ax = axs.flatten()
for i, g in enumerate(graphs):
    ax[i].set_title(g[0])
    nx.draw_networkx(g[1], pos=nx.kamada_kawai_layout(G), width=0.1 if i < 5 else 0.01, node_size=20, with_labels=False, ax=ax[i])

Lo primero que podemos notar es que, a lo largo del tiempo, las aristas crecen mucho más rápido que los nodos (ya veremos en qué proporción), y en consecuencia el diámetro se mantiene constante casi durante toda la evolución del grafo.

## Evolución macroscópica

A continuación analizaremos la relación entre la cantidad de nodos y la cantidad de aristas a lo largo del tiempo